In [6]:
import requests

# Wikipedia日本語版のAPIエンドポイント
WIKI_API_URL = "https://ja.wikipedia.org/w/api.php"

# 共通セッション（User-Agent を明示）
session = requests.Session()
session.headers.update({
    # 自分のアプリ名や連絡先URL/メールを入れるとさらに親切
    "User-Agent": "MyWikiClient/1.0 (example@example.com)"
})

# 検索キーワードに該当するWikipediaページを取得する関数
def fetch_search_results(keyword, limit=10):
    params = {
        "action": "query",
        "list": "search",
        "srsearch": keyword,
        "format": "json",
        "formatversion": 2,   # ← これで扱いやすいJSONに
        "srlimit": limit
    }

    resp = session.get(WIKI_API_URL, params=params, timeout=10)
    resp.raise_for_status()
    data = resp.json()
    return data.get("query", {}).get("search", [])

# 指定されたタイトルのページ内容を取得する関数
def fetch_page_content(title):
    params = {
        "action": "query",
        "format": "json",
        "formatversion": 2,   # ← pages が配列に
        "titles": title,
        "prop": "extracts",
        "explaintext": 1,     # ← True ではなく 1 推奨
        "redirects": 1        # ← リダイレクトも追従
    }

    resp = session.get(WIKI_API_URL, params=params, timeout=10)
    resp.raise_for_status()
    data = resp.json()
    pages = data.get("query", {}).get("pages", [])
    if not pages:
        return "ページ内容が見つかりません。"
    page = pages[0]
    return page.get("extract", "ページ内容が見つかりません。")

In [7]:
# テストコードで確認
keyword = "Python"

# キーワードに該当するページ一覧を取得
search_results = fetch_search_results(keyword)
if search_results:
    print("【キーワードに該当するページ一覧】")
    for result in search_results[:3]:
        print(f"- {result['title']}")

    # 最初のページタイトルを取得
    first_title = search_results[0]["title"]
    print("\n【最初のページタイトル】")
    print(first_title)

    # 最初のページの内容を取得
    page_content = fetch_page_content(first_title)
    print("\n【ページ内容】")
    print(page_content[:500])
else:
    print("検索結果が見つかりません。")

【キーワードに該当するページ一覧】
- Python
- モンティ・パイソン
- IronPython

【最初のページタイトル】
Python

【ページ内容】
Python（パイソン）はインタープリタ型の高水準汎用プログラミング言語である。


== 概要 ==
Pythonは1991年にグイド・ヴァン・ロッサムにより開発されたプログラミング言語である。
最初にリリースされたPythonの設計哲学は、ホワイトスペース（オフサイドルール）の顕著な使用によってコードの可読性を重視している。その言語構成とオブジェクト指向のアプローチは、プログラマが小規模なプロジェクトから大規模なプロジェクトまで、明確で論理的なコードを書くのを支援することを目的としている。
Pythonは動的に型付けされていて、ガベージコレクションされている。構造化（特に手続き型）、オブジェクト指向、関数型プログラミングを含む複数のプログラミングパラダイムをサポートしている。Pythonは、その包括的な標準ライブラリのため、しばしば「バッテリーを含む」言語と表現されている。
Pythonのインタプリタは多くのOSに対応している。プログラマーのグローバルコミュニティは、自由かつオープンソース  のリファレンス実装であるCPythonを開発および保守している 。非営利団体であるPyth


In [8]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [9]:
# 要約を行うプロンプトを作成
prompt = f"""
以下の文章を要約してください。

# 条件：
- 小学生にもわかるように
- 300文字程度

# 文章：
{page_content[:1000]}
"""

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt},
    ],
    max_tokens=500,
    temperature=0.3
)

In [10]:
# LLMからの回答を表示
print(response.choices[0].message.content.strip())

Python（パイソン）は、1991年にグイド・ヴァン・ロッサムによって作られたプログラミング言語です。この言語は、コードが読みやすくなるように設計されています。Pythonは、いろいろなプログラミングのスタイルをサポートしていて、小さなプロジェクトから大きなプロジェクトまで使えます。また、Pythonは「バッテリーを含む」と呼ばれるほど、たくさんの便利な機能が最初から用意されています。

Pythonは、動的に型付けされていて、プログラムを実行するために特別なソフトウェア（インタプリタ）が必要です。プログラマーたちは、Pythonを自由に使えるように開発し続けています。Pythonの特徴は、シンプルで効率的なコードを書くことができる点です。基本的な機能は最小限に抑えられていて、必要な追加機能はインターネットからダウンロードできます。Pythonの考え方は、「一つのことをするのに、良いやり方が一つある」というものです。
